In [ ]:
print("hello world")

hello world


# Introduction
This project follows the chess engine Giraffe


In [ ]:
import numpy as np
import pandas as pd
import os
import inspect

In [ ]:
!pip install python-chess  # Python-Chess is the Python Chess Package that handles the chess environment
!pip install --upgrade git+https://github.com/arjangroen/RLC.git  # RLC is the Reinforcement Learning package

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.1/149.1 kB 3.8 MB/s eta 0:00:00
  Cloning https://github.com/arjangroen/RLC.git to /tmp/pip-req-build-3u38nscj
  Running command git clone --filter=blob:none --quiet https://github.com/arjangroen/RLC.git /tmp/pip-req-build-3u38nscj
  Resolved https://github.com/arjangroen/RLC.git to commit e54eb7380875f64fd06106c59aa376b426d9e5ca
  Preparing metadata (setup.py) ... done
  Created wheel for RLC: filename=RLC-0.3-py3-none-any.whl size=22553 sha256=e7dfb677f68ec96c852fa2b35c460c37407026ba200c95c58593e1cc853a08bf
  Stored in directory: /tmp/pip-ephem-wheel-cache-a4jq12af/wheels/85/f1/6e/1db6247793737c4a5458c09e55ff6481f387ae570ee00a5efe
Successfully built RLC


In [ ]:
import chess
#import the testing data to test the engine that we make
with open("/content/positions.txt", "r") as f:
    lines = f.readlines()
test_data = [line[:-1] for line in lines[9::5]]
test_data


['r2qkb1r/pp2nppp/3p4/2pNN1B1/2BnP3/3P4/PPP2PPP/R2bK2R w KQkq - 1 0',
 '1rb4r/pkPp3p/1b1P3n/1Q6/N3Pp2/8/P1P3PP/7K w - - 1 0',
 '4kb1r/p2n1ppp/4q3/4p1B1/4P3/1Q6/PPP2PPP/2KR4 w k - 1 0',
 'r1b2k1r/ppp1bppp/8/1B1Q4/5q2/2P5/PPP2PPP/R3R1K1 w - - 1 0',
 '5rkr/pp2Rp2/1b1p1Pb1/3P2Q1/2n3P1/2p5/P4P2/4R1K1 w - - 1 0',
 '1r1kr3/Nbppn1pp/1b6/8/6Q1/3B1P2/Pq3P1P/3RR1K1 w - - 1 0',
 '5rk1/1p1q2bp/p2pN1p1/2pP2Bn/2P3P1/1P6/P4QKP/5R2 w - - 1 0',
 'r1nk3r/2b2ppp/p3b3/3NN3/Q2P3q/B2B4/P4PPP/4R1K1 w - - 1 0',
 'r4br1/3b1kpp/1q1P4/1pp1RP1N/p7/6Q1/PPB3PP/2KR4 w - - 1 0',
 'r1b2k1r/ppppq3/5N1p/4P2Q/4PP2/1B6/PP5P/n2K2R1 w - - 1 0',
 'r2q1b1r/1pN1n1pp/p1n3k1/4Pb2/2BP4/8/PPP3PP/R1BQ1RK1 w - - 1 0',
 '3q2r1/4n2k/p1p1rBpp/PpPpPp2/1P3P1Q/2P3R1/7P/1R5K w - - 1 0',
 'r2qk2r/pb4pp/1n2Pb2/2B2Q2/p1p5/2P5/2B2PPP/RN2R1K1 w - - 1 0',
 '6k1/pp4p1/2p5/2bp4/8/P5Pb/1P3rrP/2BRRN1K b - - 0 1',
 'rnbqkbn1/ppppp3/7r/6pp/3P1p2/3BP1B1/PPP2PPP/RN1QK1NR w - - 1 0',
 'r2qrb2/p1pn1Qp1/1p4Nk/4PR2/3n4/7N/P5PP/R6K w - - 1 0',
 '8/2r5/1k5p/1p

In [ ]:
#import the training data : consists of fen representation and the corresponding houdini evaluation values
path = "/content/train_houdini_normal_977000.csv.zip"
dataset = pd.read_csv(path)
dataset.rename(columns={'7k/p7/1bP1n1p1/4P2p/2Q2P2/2R3PK/7P/6q1 b - - 1 40': 'FEN-str', '0': 'h-eval'}, inplace=True)
print(dataset)

                                                  FEN-str  h-eval
0       5n1k/p5pp/1bP5/4P3/5P2/2R2QP1/q6P/3r3K w - - 0 35    1.44
1       3r1n1k/p5pp/1bP5/3qP3/5P2/2R2QP1/P6P/1R5K b - ...   -1.64
2       r2qk1nr/ppp2ppp/2np4/2b1p3/4PP2/2P2B2/PP1P2PP/...    0.05
3       2r2n1k/p3n1pp/1bB5/1P2P3/1q3P2/6P1/P1Q4P/3R1R1...    1.70
4       r2q1r1k/pp1n2pp/1b4n1/1P1BPp2/3P1P2/B2Q2P1/P6P...    1.87
...                                                   ...     ...
977707  rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...    0.15
977708  rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...    0.18
977709  rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...    0.15
977710  rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...    0.12
977711  rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...    0.14

[977712 rows x 2 columns]


In [ ]:
board = chess.Board(dataset["FEN-str"][0])
print(board)
#removing data with duplicate pieces and processing it to get the training data
def cleanData(fen):
  pieces = fen["FEN-str"].split()[0]
  wrong = pieces.count("Q")>1 or pieces.count("K")>1 or pieces.count("B")>2 or pieces.count("N") or pieces.count("R") or pieces.count("P")>8 or pieces.count("q")>1 or pieces.count("k")>1 or pieces.count("b")>2 or pieces.count("n") or pieces.count("r") or pieces.count("p")>8
  return not(wrong)
dataset_new = dataset[dataset.apply(cleanData, axis = 1)]
dataset_final = dataset_new.sample(10000)
dataset_final.reset_index(inplace=True)
dataset_final.drop(["index"], axis =1)

. . . . . n . k
p . . . . . p p
. b P . . . . .
. . . . P . . .
. . . . . P . .
. . R . . Q P .
q . . . . . . P
. . . r . . . K


,FEN-str,h-eval
0,8/7p/5Kp1/8/3k4/7P/6P1/8 b - - 2 54,-10.56
1,8/8/2Q2k1P/2p5/2P3K1/8/4P3/3q4 b - - 6 73,-16.48
2,8/pp2qkp1/1b2p2p/3p4/P4Q2/4P1P1/1P3PBP/6K1 b -...,0.00
3,5qk1/7p/6pB/5p2/1p6/1P4PP/5PK1/8 w - - 0 44,1.51
4,8/3k4/7p/1bPB1p1P/3p4/3Pb1B1/4K3/8 w - - 15 73,0.01
...,...,...
9995,8/8/pp2p1pB/1k2bp2/4P1P1/1P1K1P2/P7/8 w - - 6 40,0.03
9996,6k1/5p2/8/5PP1/1q3QK1/8/8/8 b - - 8 64,-0.40
9997,8/6kp/p1b1pp2/7P/2p3P1/P1PpB3/5P2/2K5 w - - 0 30,-0.50
9998,2B5/8/1Pb2k2/4bpp1/8/8/5B2/5K2 b - - 5 41,0.59


In [ ]:
print(dataset_final["FEN-str"][9000])

8/3b4/pp6/2p2p1k/2P1pP2/P3P1K1/4B3/8 b - - 1 44


We follow the FEN representation. The FEN representation basically shows a position on the board and gives info about the play


1.   Field 1 : positions of pieces on the board, upper case for White lower case for Black
2.   Field 2 : w if white is to move and b if black is to move
3.   Field 3 : Castling availability, - if no castling, K if white can castle kingside, Q if white can castle queenside and k and q for black.
4.   Field 4 : Square over which pawn has passed while moving 2 squares
5.   Field 5 : Halfmove Clock
6.   Field 6 : Fullmove number


In [ ]:
#we have a dataset of fen representations and the houdini evaluation. This will be the training dataset.
#Based on the giraffe paper, the feature representation is in 3 groups. The first is global features

def globalfeat(a):
  fen = a if isinstance(a, str) else a["FEN-str"]
  fields = fen.split()
  board = chess.Board(fen)
  k,K,q,Q = 0,0,0,0
  if fields[1] == 'w' : whiteMove = 1
  else: whiteMove = 0
  if 'K' in fields[2] :
    K = 1
  elif 'k' in fields[2] :
    k = 1
  elif 'Q' in fields[2] :
    Q = 1
  elif 'q' in fields[2] :
    q = 1
  return [whiteMove, K, Q, k, q]


In [ ]:
# The second component of feature representation is the number of each piece and the existence and position of each piece

def piecewisefeat(a):
  fen = a if isinstance(a, str) else a["FEN-str"]
  piececount = [0]*10 # [Q,B,N,R,P,q,b,n,r,p]
  piecepos = [0]*32 #[Kexists, Kpos, kexists, kpos, Qe, Qp, qe, qp, B1e, B1p,B2e, B2p,etc b1e, b1p, N1e, N1p, n1e, n2e, R1e, R2e, r1e, r2e ]
  piecepawnw = []
  piecepawnb = []

  board = chess.Board(fen)
  pieces_map = chess.Board.piece_map(board)
  wking_square_index = board.king(chess.WHITE)
  wking_square_name = chess.square_name(wking_square_index)
  bking_square_index = board.king(chess.BLACK)
  bking_square_name = chess.square_name(bking_square_index)
  #print(pieces_map)
  piecepos[0],piecepos[1],piecepos[2],piecepos[3] = 1, wking_square_name, 1, bking_square_name
  fields = fen.split()

  for l,i in pieces_map.items():
    #print(l,i)
    if i == chess.Piece.from_symbol("Q"):
      piececount[0] +=1
      piecepos[4] = 1
      piecepos[5] = chess.square_name(l)
    if i == chess.Piece.from_symbol("q"):
      piececount[5] +=1
      piecepos[6] = 1
      piecepos[7] = chess.square_name(l)
    if i == chess.Piece.from_symbol("B"):
      piececount[1]+=1
      if piececount[1] == 1:
        piecepos[8] = 1
        piecepos[9] = chess.square_name(l)
      elif piececount[1] == 2:
        piecepos[10] = 1
        piecepos[11] = chess.square_name(l)
    if i == chess.Piece.from_symbol("b"):
      piececount[6]+=1
      if piececount[6] == 1:
        piecepos[12] = 1
        piecepos[13] = chess.square_name(l)
      elif piececount[6] == 2:
        piecepos[14] = 1
        piecepos[15] = chess.square_name(l)
    if i == chess.Piece.from_symbol("N"):
      piececount[2]+=1
      if piececount[2] == 1:
        piecepos[16] = 1
        piecepos[17] = chess.square_name(l)
      elif piececount[2] == 2:
        piecepos[18] = 1
        piecepos[19] = chess.square_name(l)
    if i == chess.Piece.from_symbol("n"):
      piececount[7]+=1
      if piececount[7] == 1:
        piecepos[20] = 1
        piecepos[21] = chess.square_name(l)
      elif piececount[7] == 2:
        piecepos[22] = 1
        piecepos[23] = chess.square_name(l)
    if i == chess.Piece.from_symbol("R"):
      piececount[3]+=1
      if piececount[3] == 1:
        piecepos[24] = 1
        piecepos[25] = chess.square_name(l)
      elif piececount[3] == 2:
        piecepos[26] = 1
        piecepos[27] = chess.square_name(l)
    if i == chess.Piece.from_symbol("r"):
      piececount[8]+=1
      if piececount[8] == 1:
        piecepos[28] = 1
        piecepos[29] = chess.square_name(l)
      elif piececount[8] == 2:
        piecepos[30] = 1
        piecepos[31] = chess.square_name(l)
    if i == chess.Piece.from_symbol("P"):
      piececount[4]+=1
      piecepawnw.append(1)
      piecepawnw.append(chess.square_name(l))
    if i == chess.Piece.from_symbol("p"):
      piececount[9]+=1
      piecepawnb.append(1)
      piecepawnb.append(chess.square_name(l))
  tempw = [0]*(16 - len(piecepawnw))
  tempb = [0]*(16 - len(piecepawnb))
  piecepawnw.extend(tempw)
  piecepawnb.extend(tempb)
  piecespos = piecepos+piecepawnw+piecepawnb
  return piececount+piecespos

print(piecewisefeat("8/3b4/pp6/2p2Q1k/2P1pP2/P3P1K1/4B3/8 b - - 1 44"))
print(chess.Board("8/3b4/pp6/2p2Q1k/2P1pP2/P3P1K1/4B3/8 b - - 1 44"))

[1, 1, 0, 0, 4, 0, 1, 0, 0, 4, 1, 'g3', 1, 'h5', 1, 'f5', 0, 0, 1, 'e2', 0, 0, 1, 'd7', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 'f4', 1, 'c4', 1, 'e3', 1, 'a3', 0, 0, 0, 0, 0, 0, 0, 0, 1, 'b6', 1, 'a6', 1, 'c5', 1, 'e4', 0, 0, 0, 0, 0, 0, 0, 0]
. . . . . . . .
. . . b . . . .
p p . . . . . .
. . p . . Q . k
. . P . p P . .
P . . . P . K .
. . . . B . . .
. . . . . . . .


In [ ]:
#The third component of feature representation is the square centric features : attack and defend maps

def scfeat(a):
  fen = a if isinstance(a, str) else a["FEN-str"]
  board = chess.Board(fen)
  val = {6:1, 5:9, 3:3, 2:3, 4:5, 1:1}
  w = []
  b = []
  wd=[]
  bd=[]
  #attack map
  for sq in chess.SQUARES:
      wattacks = chess.BaseBoard.attackers(board, chess.WHITE, sq)
      battacks = chess.BaseBoard.attackers(board, chess.BLACK, sq)
      w.append(0 if len(wattacks)==0 else min(val[chess.BaseBoard.piece_type_at(board, x)] for x in wattacks))
      b.append(0 if len(battacks)==0 else min(val[chess.BaseBoard.piece_type_at(board, x)] for x in battacks))
  #defend map
  return (w+b)
print(scfeat("8/3b4/pp6/2p2Q1k/2P1pP2/P3P1K1/4B3/8 b - - 1 44"))
print(len(scfeat("8/3b4/pp6/2p2Q1k/2P1pP2/P3P1K1/4B3/8 b - - 1 44")))

[0, 0, 0, 3, 0, 3, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 3, 0, 1, 0, 1, 0, 1, 3, 1, 9, 1, 1, 1, 0, 1, 9, 1, 1, 0, 1, 3, 0, 0, 0, 0, 9, 9, 9, 0, 0, 0, 0, 9, 0, 9, 0, 9, 0, 0, 0, 0, 0, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 3, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 3, 1, 0, 0, 0, 3, 0, 3, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 3, 0, 0, 0]
128


In [ ]:
#Now we have the 3 components for feature representation. Pre process and convert this into appropriate featurised input to the deep q network

def featureExtraction(data):
  global_col = ["white move", "white kingside castle", "white queenside castle","black kingside castle", "black queenside castle"]
  attackmap_col = ["white attack" + str(i) for i in range(64)] + ["black attack" + str(i) for i in range(64)]
  pieces_col = ["cW","cB","cN","cR","cP","cw","cb","cn","cr","cp", "Ke","Kp","ke","kp", "Qe","Qp","qe","qp",
                "B1e","B1p","B2e","B2p","b1e","b1p","b2e","b2p","N1e","N1p","N2e","N2p","n1e","n1p","n2e","n2p",
                "R1e","R1p","R2e","R2p","r1e","r1p","r2e","r2p","P1e","P1p","P2e","P2p","P3e","P3p","P4e","P4p",
                "P5e","P5p","P6e","P6p","P7e","P7p","P8e","P8p","p1e","p1p","p2e","p2p","p3e","p3p","p4e","p4p",
                "p5e","p5p","p6e","p6p","p7e","p7p","p8e","p8p"]
  print(len(pieces_col))
  count_col = ["cW","cB","cN","cR","cP","cw","cb","cn","cr","cp"]
  #now we make dataframes
  feat_global = pd.DataFrame()
  feat_map = pd.DataFrame()
  feat_pieces = pd.DataFrame()

  feat_global[global_col] = data[["FEN-str"]].apply(globalfeat, axis=1, result_type="expand")
  feat_map[attackmap_col] = data[["FEN-str"]].apply(scfeat, axis=1, result_type="expand")
  feat_pieces[pieces_col] = data[["FEN-str"]].apply(piecewisefeat, axis=1, result_type="expand")
  feat_global = feat_global.join(feat_pieces[count_col])
  feat_pieces.drop(count_col, axis=1, inplace=True)

  y = data["h-eval"]
  return feat_global, feat_pieces, feat_map, y

In [ ]:
#Now we extract features from out training dataset
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
f1,f2,f3,y = featureExtraction(dataset_final)
print(f1, f2, f3)

74
      white move  white kingside castle  white queenside castle  \
0              0                      0                       0   
1              0                      0                       0   
2              0                      0                       0   
3              1                      0                       0   
4              1                      0                       0   
...          ...                    ...                     ...   
9995           1                      0                       0   
9996           0                      0                       0   
9997           1                      0                       0   
9998           0                      0                       0   
9999           0                      0                       0   

      black kingside castle  black queenside castle  cW  cB  cN  cR  cP  cw  \
0                         0                       0   0   0   0   0   2   0   
1                         0       